Финальное задание по практике
--------------------

### Цель

Ознакомится с базовыми алгоритмами и базовыми инструментами машинного обучения для анализа текста

### Задание

1) Получить категоризованные данные с <b>Facebook API</b><br>
2) Сохранить полученные данные<br>
3) Токенизация данных<br>
4) Удаление стоп-слов<br>
5) Стемминг<br>
6) Поиск паттерна имя-фамилия в тексте<br>
7) Формирование списка топ <i>100</i> важных токенов по каждой категории<br>
8) Категоризация новых текстов по наличию в них этих топ <i>100</i> токенов<br>
9) Оценка качества<br>

Кодировка для кода на <b>Python</b>

In [27]:
# coding=utf-8

Для измирения времени работы некоторых участоков подключим модуль <b>"time"</b>

In [28]:
import time as t

# вернет: текущее время в миллисекундах
def time(): return int(round(t.time() * 1000))

## Хранение данных

Для хранения данных была выбрана база данных <b>Elasticsearch</b>, которая предоставляет удобный интерфейс взаимодействия, а так же кроссплатформенность при работе с ней, за счет того, что взаимодействие осуществляется по HTTP протоколу.

Константы работы с базой данных <b>Elasticsearch</b>:

In [29]:
ES_INDEX = "fb_group_posts"                 # Аналог базы данных в сравнении с реляционными БД, под ним будут
                                            # хранится все данные, необходимые по заданию
ES_POST_DOC_TYPE = "post"                   
ES_NAME_RELATION_DOC_TYPE = "name_relation"

ES_BULK_ACTIONS_SIZE = 500                  # Размер пака данных отсылаемых за раз в elasticsearch, 
                                            # необходимо для оптимизации по скорости

Работа с базой данных не относится к заданию напрямую, по-этому описанию работе с ней уделено меньше внимания. Вся работа с <b>Elasticsearch</b> скрыта в ниже описанном классе <b>FacebookDBHelper</b>.

In [30]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import TransportError
from elasticsearch import helpers
from copy import deepcopy


# Дата-класс, который описывает отношение наличия имени в тексте поста
# @param: fl_name - Имя Фамилия
# @param: post_id - id поста, в тексте которого содержится это имя и фамилия
class FLNameData(object):
    def __init__(self, fl_name, post_id):
        self.fl_name = fl_name
        self.post_id = post_id


class FacebookDBHelper(object):
    def __init__(self):
        self.es = Elasticsearch()
    
    def save_posts(self, group_name, group_domain, posts):
        actions = []

        for post in posts:
            if 'message' in post.keys():
                action = {
                    "_index": ES_INDEX,
                    "_type": ES_POST_DOC_TYPE,
                    "_id": post['id'],
                    "_source": {
                        "message": post['message'],
                        "group_name": group_name,
                        "group_domain": group_domain
                    }
                }

                actions.append(action)

        self.__bulk_insert(actions)
        
    def save_name_relations(self, relations):
        actions = []

        for relation in relations:
            action = {
                "_index": ES_INDEX,
                "_type": ES_NAME_RELATION_DOC_TYPE,
                "_source": {
                    "fl_name": relation.fl_name,
                    "post_id": relation.post_id
                }
            }

            actions.append(action)

        self.__bulk_insert(actions)

    def __bulk_insert(self, actions):
        actions_list = split_list(list(actions), ES_BULK_ACTIONS_SIZE)

        for acts in actions_list:
            helpers.bulk(self.es, acts)

    def get_post_by_id(self, id):
        return self.__get(doc_type=ES_POST_DOC_TYPE, id=id)

    def get_all_posts(self):
        return self.__get_all(doc_type=ES_POST_DOC_TYPE)

    def get_all_name_relations(self, doc_type):
        relations = self.__get_all(doc_type=doc_type)

        # noinspection PyTypeChecker
        return [FLNameData(fl_name=r['_source']['fl_name'], post_id=r["_id"]) for r in relations]

    def __get(self, doc_type, id):
        return self.es.get(index=ES_INDEX, doc_type=doc_type, id=id)

    def __get_all(self, doc_type, body=None):
        if body is None:
            body = {}

        result = []

        page = self.es.search(
            index=ES_INDEX,
            doc_type=doc_type,
            scroll='2m',
            search_type='scan',
            size=1000,
            body=body)

        scroll_id = page['_scroll_id']
        scroll_size = page['hits']['total']

        while scroll_size > 0:
            page = self.es.scroll(scroll_id=scroll_id, scroll='2m')

            scroll_id = page['_scroll_id']
            scroll_size = len(page['hits']['hits'])

            result.extend(page['hits']['hits'])

        return result

    def get_all_sources(self, doc_type):
        posts = self.get_all_posts()
        result = []

        for post in posts:
            if "_source" in post.keys():
                # noinspection PyTypeChecker
                result.append(post["_source"])

        return result

    def get_all_messages(self):
        sources = self.get_all_sources(doc_type=ES_POST_DOC_TYPE)
        result = []

        for source in sources:
            if 'message' in source.keys():
                # noinspection PyTypeChecker
                result.append(source["message"])

        return result

    def delete_all_posts(self):
        return delete_by_doc_type(
            es=self.es,
            index=ES_INDEX,
            type_=ES_POST_DOC_TYPE)

    def delete_all_name_relations(self):
        return delete_by_doc_type(
            es=self.es,
            index=ES_INDEX,
            type_=ES_NAME_RELATION_DOC_TYPE)

    def get_messages_by_domain(self, domain):
        posts = self.__get_all(doc_type=ES_POST_DOC_TYPE, body={
            "query": {
                "match": {
                    "group_domain": {
                        "query": domain,
                        "operator": "and"
                    }
                }
            }
        })
        
        return [p['_source']['message'] for p in posts]

    def get_name_relations_by_fl(self, fl_name):
        return self.__get_all(doc_type=ES_NAME_RELATION_DOC_TYPE, body={
            "query": {
                "match": {
                    "fl_name": {
                        "query": fl_name,
                        "operator": "and"
                    }
                }
            }
        })
    
    def get_all_domains(self):
        response = self.es.search(index=ES_INDEX, doc_type=ES_POST_DOC_TYPE, body={
            "size": 0,
            "aggs": {
                "langs": {
                    "terms": {
                        "field": "group_domain",
                    }
                }
            }
        })

        result = []

        for bucket in response['aggregations']['langs']['buckets']:
            result.append(bucket['key'])

        return result


def delete_by_doc_type(es, index, type_):
    try:
        count = es.count(index, type_)['count']
        max_count = 5000

        if not count:
            return 0

        tmp_count = count

        while tmp_count > 0:
            tmp_count -= max_count

            response = es.search(
                index=index,
                filter_path=["hits.hits._id"],
                body={"size": max_count,
                      "query": {
                          "filtered": {
                              "filter": {
                                    "type": {"value": type_}
                              }
                          }
                      }})

            if not response:
                return 0

            ids = [x["_id"] for x in response["hits"]["hits"]]

            if not ids:
                return 0

            bulk_body = [
                '{{"delete": {{"_index": "{}", "_type": "{}", "_id": "{}"}}}}'.format(index, type_, x)
                for x in ids]

            es.bulk('\n'.join(bulk_body))
            es.indices.flush_synced([index])

        return count
    except TransportError as ex:
        print("Elasticsearch error: " + ex.error)
        raise ex
        
def split_list(list_, count_):
    result = []

    if len(list_) == 0:
        return result

    if len(list_) == count_:
        result.append(list_)

        return result

    steps = len(list_) / count_
    tmp_list = deepcopy(list_)

    for i in range(0, steps):
        result.append(tmp_list[0: count_])
        tmp_list = tmp_list[count_: len(tmp_list)]

    if len(tmp_list) != 0:
        result.append(tmp_list)

    return result


Экземпляр класса <b>FacebookDBHelper</b>, через который мы будем взаимодействовать с <b>Elasticsearch</b> во всей работе

In [31]:
fdb = FacebookDBHelper()

## Загрузка постов с facebook.com используя Gaph API

Константы для работы с <b>Facebook API</b>

In [32]:
# Токен зарегестрированного приложения в консоли разработчика на facebook.com
FACEBOOK_TOKEN = "1769775703259571|736fc7f9c5dc31707d40709a1d37813b"

# Кол-во постов загружаемых с одной группы на facebook.com
FACEBOOK_POSTS_COUNT = 3000

Сущьность описывающая заведомо известные данные о группе в <b>Facebook</b>:
* Имя
* Id на facebook.com
* Домен (класс) тематики группы

In [33]:
class Group(object):
    def __init__(self, name, id, domain):
        self.name = name
        self.id = id
        self.domain = domain

Список групп, которые будут использованы для работы (с которых мы будем загружать посты)

In [34]:
groups = [
#     Group(name="CNN Politics", id="219367258105115", domain="politics"),
#     Group(name="SinoRuss", id="1565161760380398", domain="politics"),
#     Group(name="Politics & Sociology", id="1616754815303974", domain="politics"),
#     Group(name="CNN Money", id="6651543066", domain="finances"),
#     Group(name="MTV", id="7245371700", domain="music"),
#     Group(name="CNET", id="7155422274", domain="tech"),
#     Group(name="TechCrunch", id="8062627951", domain="tech"),
#     Group(name="Sport Addicts", id="817513368382866", domain="sport"),
#     Group(name="Pokemon GO", id="1745029562403910", domain="pokemon_go")
]

Для работы с Facebook будем использовать <b>Graph API</b>, это высокоуровневый <i>HTTP</i> клиент

In [35]:
from facebook import GraphAPI


graph = GraphAPI(access_token=FACEBOOK_TOKEN)

Так же нам понадобится модуль для совершения <b>HTTP</b> запросов. Необходим для тех действий, которые недоступны в <b>Graph API</b>

In [36]:
import requests

Функция загружающая последовательно посты пока не достигнет предела группы или ограничения по кол-ву (константа <b>FACEBOOK_POSTS_COUNT</b>)

In [37]:
def load_next_posts(posts, max_count):
    result = []
    count = 0

    while True:
        if count > max_count: # Если зугрузили необходимое кол-во то прекращаем работу
            break

        try:
            for post_data in posts['data']: # Под ключем 'data' хранится список постов в паке
                keys = post_data.keys()     # Так как мы нам нужны сами сообщения
                                            
                
                if 'message' in keys:       # Добавляем в результат только те у которых есть
                    result.append(post_data)# текстовое сообщение (смотрим поналичию ключа 'message')

            # Меняем размер запрашиваемого пака (страницы) с постами, в уже сформированном запросе 
            # от Facebook Graph API.
            request = posts['paging']['next'].replace("limit=25", "limit=100")

            # Выполняем запрос на получение следующей страницы с вопросами
            s_time = time()
            posts = requests.get(request).json()
            f_time = time()
            
            posts_count = len(posts['data'])
            count += posts_count
            
            print "Время загрузки пака постов ->", (f_time - s_time), "|", "кол-во:", posts_count, "|", "всего:", count
        except KeyError:
            break

    print "Общее кол-во загруженных постов группы ->", count

    return result

Используя ранее описанный список постов заполняем нашу базу данных

In [38]:
for group in groups:
    # Получаем данные о группе по ее id на facebook.com
    s_time = time()
    group_json = graph.get_object(group.id)
    f_time = time()
    
    print 'Время загрузки данных [', group_json['name'], '] группы ->', (f_time - s_time)
    
    # Получаем первый пак постов, используем уже для этого метод "get_connections"
    # который, грубо говоря, дает возможность запрашивать списки
    s_time = time()
    first_posts_pack = graph.get_connections(group_json['id'], 'feed')
    f_time = time()
    
    print 'Время загрузки первого пака постов ->', (f_time - s_time)
    
    # Последовательно загружаем все последующие посты
    posts = load_next_posts(posts=first_posts_pack, max_count=FACEBOOK_POSTS_COUNT)
    
    # Сохраняем все в базу данных
    s_time = time()
    fdb.save_posts(group.name, group.domain, posts)
    f_time = time()
    
    print "Время сохранение постов в БД ->", (f_time - s_time)
    print "-------------------------"

## Поиск паттерна (шаблона) "Имя Фамилия"

Решение в лоб - это при поступление запроса <b>"имя_фамилия"</b> на поиск просматривать весь список постов на наличе подстроки в их тексте шаблона <b>"Имя Фамилия"</b>. Главная проблема такого решения это скорость работы, так как объемы даных в которых будет осуществлятся поиск скорее всго будет достаточно большим.

Мое решение заключается в следующем:<br>
Подготовка данных, в которых будет осуществлятся поиска, перед самим поиском. Суть в том, чтобы найти все пары "Имя фамилия" зарание и единожды, сохранив отношения <b>"имя фамилия" <==> "пост в котором было найдено имя фамилия"</b>. А сам поиск будем осуществлять уже в этом подготовленном списке отношений.

In [39]:
import re

# Регулярное выражение для "Имя Фамилия"
FIRST_LAST_NAME_PATTERN = "[A-Z]{1}[a-z]+\s+[A-Z]{1}[a-z]+"

Далее будем пользоваться уже сохраненными данными (текст постов, которые мы сохранили в базу данных <b>Elasticsearch</b>).

In [40]:
posts = fdb.get_all_posts() # Вытаскиваем все посты

Теперь отищем все имена в тексте каждого поста и сразу же сохраним их в виде отношений в базу данных

In [41]:
name_relations = []

s_time = time()

for post in posts:
    message = post['_source']['message']
    names = re.findall(FIRST_LAST_NAME_PATTERN, message)

    if names:
        id = post['_id']

        for name in names:
            name = name.replace(".", "")
            relation = FLNameData(fl_name=name, post_id=id)
            name_relations.append(relation)

f_time = time()

print "Время поиска имен по регулярному выражению ->", (f_time - s_time)
print "Кол-во найденных совпадений по регулярному выраженияю:", len(name_relations)

Время поиска имен по регулярному выражению -> 244
Кол-во найденных совпадений по регулярному выраженияю: 30340


Поскольку все отношения мы так же будем хранить в базе данных, то очистим ранее созданные связи

In [42]:
s_time = time()
deleted_posts = fdb.delete_all_name_relations()
f_time = time()

print "Время удаления старых отношений 'сообщение <=> имя' ->", (f_time - s_time)
print "Удалено старых отношений:", deleted_posts

Время удаления старых отношений 'сообщение <=> имя' -> 11436
Удалено старых отношений: 60605


In [43]:
s_time = time()
fdb.save_name_relations(name_relations)
f_time = time()

print "Время сохранения новых отношений 'текст поста <=> имя' ->", (f_time - s_time)

Время сохранения новых отношений 'текст поста <=> имя' -> 13640


Когда все связи сохранены, мы можем осуществить поиск.


Ход действий таков:
1. Сделать запрос в базу данных, что бы достать все свзязи в которых имя совпадает с запрашиваемым
2. Получить id групп из результата (пункт 1)
3. По найденным id групп мы теперь можем получить посты, тексты которых содержат запрашиваемые имена.

Для демонстрации мы возьмем предположительно самое популярное имя <i>"Donald Trump"</i> (имя, которое не сходит с уст всевозможных СМИ), так как большинство сохраненных постов относятся к домену "Политика"

In [50]:
# Список имен для поиска
searched_names = [
    "John Kerry",
    "Paul Reichler",
    "Donald Trump"
]

def find_messages_by_fl_name(fl_name):
    finded_relations = fdb.get_name_relations_by_fl(fl_name=fl_name)
    messages = []
    finded_post_ids = set() # Необходимо, что бы учитывать ранее найденные посты

    for relation in finded_relations:
        post_id = relation['_source']['post_id']

        if not post_id in finded_post_ids:
            finded_post_ids.add(post_id)
            
            message = fdb.get_post_by_id(post_id)['_source']['message']
            messages.append(message)

    return messages


for name in searched_names:
    messages = find_messages_by_fl_name(name)
    
    print "Имя [", name, "] найдено соответстивий (сообщений):", len(messages)

Имя [ John Kerry ] найдено соответстивий (сообщений): 19
Имя [ Paul Reichler ] найдено соответстивий (сообщений): 10
Имя [ Donald Trump ] найдено соответстивий (сообщений): 185


Так же, частью подготовки документов (постов с facebook) является удаление стоп-слов (stop-words), это слова которые не несут никакой значимой информации и никак не отображают тему текста. В русском это были бы слова "а", "и", "или", "в" и тд. Эти слова встречаются в всюду и избавившись от них мы сделаем наши данные чище.

Список стоп-слов для английского языка лежат в отдельном файле с иминем "stop-words.txt". 

In [51]:
def load_stop_words(file_name):
    words = set()

    f = open(file_name, 'r')

    for line in f:
        words.add(line.strip())

    f.close()

    return words

stop_words = load_stop_words("stop_words.txt")
print "Размер словаря стоп-слов:", len(stop_words), "слова"

Размер словаря стоп-слов: 323 слова


Пакет <b>scikit-learn</b> уже содержит список стоп-слов для английского языка, и что бы не упустить ничего, объеденим их словарь с нашим (тот, который мы выгрузили с файла ранее)

In [52]:
from sklearn.feature_extraction import text 


stop_wrods = text.ENGLISH_STOP_WORDS.union(stop_words)
print "Размер объедененного словаря стоп-слов:", len(stop_words), "слова"

Размер объедененного словаря стоп-слов: 323 слова


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem.porter import *


class StemmedCountVectorizer(CountVectorizer):
    def __init__(self):
        super(StemmedCountVectorizer, self).__init__()
        self.stemmer = PorterStemmer()  # проинициализируем стиммер Портера
        self.stop_words = stop_words    # проинициализируем словарь стоп-слов

    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc:(self.stemmer.stem(w) for w in analyzer(doc))


def create_vectorizer(): return StemmedCountVectorizer()

Выполним запрос к базе данных, что бы вытащить списко всех доменов (тех, что мы записывали в базу данных)

In [56]:
domains = fdb.get_all_domains()

Теперь создадим словарь, где ключем будет служить имя домена, а значением посты по этому домену.

In [57]:
s_time = time()
posts = dict([(domain, fdb.get_messages_by_domain(domain)) for domain in domains])
f_time = time()

print "Время получения постов с базы данных ->", (f_time - s_time)

Время получения постов с базы данных -> 416


In [58]:
for k, v in posts.items():
    print "Кол-во постов [", "{0: <10}".format(str(k)), "]:", len(v)

Кол-во постов [ pokemon_go ]: 338
Кол-во постов [ music      ]: 3009
Кол-во постов [ tech       ]: 5467
Кол-во постов [ politics   ]: 5576
Кол-во постов [ sport      ]: 1092
Кол-во постов [ finances   ]: 2652


Что бы работать с каждым доменом независимо и в дальнейшем иметь доступ к их словарям, создаем для каждого домена свой векторизатор. Векторизаторы (в данном случае CountVectorizer) будут хранится в словаре, под ключем имени домена.

In [ ]:
# Словарь => key="имя_домена", value="векторизатор"
vs = dict([(domain, create_vectorizer()) for domain in domains])

Теперь векторизируем с помощью векторизатора соответствующий корпус.

In [61]:
# Словарь => key="имя_домена", value="матрица векторов соответствующего корпуса"
xs = dict([(domain, vs[domain].fit_transform(posts[domain])) for domain in domains])

In [62]:
for k, v in vs.items():
    print "Размеры словарей [", "{0: <10}".format(str(k)), "]:", len(v.vocabulary_)

Размеры словарей [ pokemon_go ]: 1038
Размеры словарей [ music      ]: 3279
Размеры словарей [ tech       ]: 7892
Размеры словарей [ politics   ]: 34597
Размеры словарей [ sport      ]: 3074
Размеры словарей [ finances   ]: 6907


In [107]:
def find_top_tokens(matrix, vocabulary, amount, domain_name=None):
    s_time = time()
    
    counts = [(word, matrix.getcol(col_num).sum()) for word, col_num in vocabulary.items()]
    tokens = sorted(counts, key = lambda x: -x[1])[:min(amount, len(counts))]
    tokens_res = []
    
    for i in range(len(tokens)):
        token = tokens[i]
        tokens_res.append((token[0], i))
    
    f_time = time()
    
    if domain_name == None:
        print "Время поиска [", amount, "] популярных токенов ->", (f_time - s_time)
    else:
        print "Время поиска [", "{0: <10}".format(domain_name), "-", amount, "] популярных токенов ->", (f_time - s_time)
        
    return tokens_res

In [132]:
top_words_dict = dict([(d, find_top_tokens(xs[d], vs[d].vocabulary_, 100, d)) for d in domains])

Время поиска [ politics   - 100 ] популярных токенов -> 54690
Время поиска [ tech       - 100 ] популярных токенов -> 3021
Время поиска [ music      - 100 ] популярных токенов -> 672
Время поиска [ finances   - 100 ] популярных токенов -> 1721
Время поиска [ sport      - 100 ] популярных токенов -> 412
Время поиска [ pokemon_go - 100 ] популярных токенов -> 76


In [133]:
top_words_set = dict([(d, set(w[0] for w in top_words_dict[d])) for d in domains])
len(top_words_set)

6

In [134]:
for k, v in top_words_dict.items():
    print "Несколько популярных слов со списка [", "{0: <10}".format(str(k)), "]:"
    print v[:min(len(v), 5)], "\n"

Несколько популярных слов со списка [ pokemon_go ]:
[(u'pokemon', 0), (u'app', 1), (u'gym', 2), (u'play', 3), (u'catch', 4)] 

Несколько популярных слов со списка [ music      ]:
[(u'new', 0), (u'like', 1), (u'mtv', 2), (u'live', 3), (u'make', 4)] 

Несколько популярных слов со списка [ tech       ]:
[(u'new', 0), (u'make', 1), (u'want', 2), (u'like', 3), (u'appl', 4)] 

Несколько популярных слов со списка [ politics   ]:
[(u'china', 0), (u'news', 1), (u'trump', 2), (u'say', 3), (u'media', 4)] 

Несколько популярных слов со списка [ sport      ]:
[(u'team', 0), (u'sport', 1), (u'like', 2), (u'nfl', 3), (u'best', 4)] 

Несколько популярных слов со списка [ finances   ]:
[(u'cnnmon', 0), (u'cnntech', 1), (u'year', 2), (u'new', 3), (u'trump', 4)] 



In [180]:
docs_test = [
    "As islanders look forward to next year, many say they hope — and even expect — Hillary Clinton to spend at least part of her summer vacations on Martha's Vineyard if she becomes president.",
    "The Pentagon says staff can still play the game on their personal phones.",
    "Ronald Reagan's daughter Patti Davis is citing her father's shooting as evidence that comments like Donald J. Trump's recent blast against Hillary Clinton have real-world consequences.",
    "If Hillary Clinton wins in November, she will be the first former secretary of state to take over the Oval Office since 1857.",
    "'The Second Amendment was put in there not just so we can go shoot skeet or go shoot trap. It was put in so we could defend our First Amendment, the freedom of speech, and also to defend ourselves against our own government,' says US Olympic skeet shooter Kim Rhode.",
    "According to the Pentagon, Special Operation Forces targeted and killed ISIS leader Hafiz Sayed Khan in Afghanistan.",
    "pokemon cool game play"
]

In [ ]:
import operator

# Поиск элементов в итерируемых коллекциях
def find_elm(iterable, predicate_):
    for v in iterable:
        if predicate_(v):
            return v

    return None

# Для поиска категории к которой относится текст (функция find_domain(txt)), 
# нам понядобится один вектооризатор
fd_vec = create_vectorizer()

def find_domain(txt):
    # Если список категорий пуст, то нечего искать
    if len(domains) == 0:
        return None
    
    # Так векторизатор перезапишет свое состояние и можно будет получить список слов текста,
    # в которых удалены стоп слова и выполнен стемминг
    fd_vec.fit_transform([txt])
    
    # Теперь используя словарь данного текста - превратим его в сет,
    # что необходимо для последующих манипуляций
    words = [item[0] for item in fd_vec.vocabulary_.items()]
    
    # Сет слов текста
    words_set = set(words)
    
    # Тут мы будем хранить пересечения словаря этого текста со словарями 
    # по каждой категории
    res_buffer = []
    
    words_intersection_set = None
    
    # Сюда будем складывать результаты пересечений множеств слов текста и топ 100 по категориям
    isds = []
    
    # Для каждого домена со списка
    for domain in domains:
        # Достаем для данного домена сет топ 100-та слов
        domain_top_words_set = top_words_set[domain]
        
        # Находим пересечение словарей по категориям и словаря текста
        words_intersection_set = words_set.intersection(domain_top_words_set)    
        words_intersection_len = len(words_intersection_set)
        
        isds.append((domain, words_intersection_set))
        res_buffer.append((domain, words_intersection_len))
    
    if len(res_buffer) == 1:
        return res_buffer[0][0]

    # Находим категорию с наибольшим кол-во пересечений
    top_tuple = max(res_buffer, key=lambda t: t[1])
    # Максимальное кол-во пересечений
    max_count = (top_tuple)[1]
    
    # Формируем список из категорий с которыми одинаковое кол-во пересечений
    res = [name for name, count in res_buffer if count == max_count]
    
    if len(res) == 0:
        return None
    
    if len(res) == 1:
        return res[0]
    
    # Имя результирующей категории (домена)
    domain_res = None
    # Вес результирующей категории (домена)
    domain_vol = -1
    
    for domain in res:
        # Находим вес для каждого слова в пересечениях с разными категориями
        # чтобы по весу сказать какая категория преобладает
        top_words = top_words_dict[domain]
        
        # Вес
        vol = 0
        
        words_intersection_set = find_elm(isds, lambda e: e[0] == domain)[1]
        
        for w in words_intersection_set:
            vol_tmp = find_elm(top_words, lambda e: e[0] == w)[1]
            
            if vol_tmp != None:
                vol += vol_tmp
        
        # В случае, если и вес одинаков, сравниваем имена груп
        # и выбор делаем в сторону большей
        if vol > domain_vol:
            domain_res = domain
            domain_vol = vol
        elif vol == domain_vol:
            if domain > domain_res:
                domain_res = domain
                domain_vol = vol
    
    return domain_res

        
for doc in docs_test:
    domain = find_domain(doc)  
    print doc[0:45] + "...", "->", domain 

As islanders look forward to next year, many ... -> politics
The Pentagon says staff can still play the ga... -> tech
Ronald Reagan's daughter Patti Davis is citin... -> finances
If Hillary Clinton wins in November, she will... -> finances
'The Second Amendment was put in there not ju... -> music
According to the Pentagon, Special Operation ... -> politics
pokemon cool game play... -> pokemon_go
